# Imports

In [1]:
from morfeus import BuriedVolume
import numpy as np
import pandas as pd
import statistics
import matplotlib.pyplot as plt
from matplotlib import rcParams
import os, re
from pandas import ExcelWriter
import xlsxwriter
import get_sum_conformers_v3 as geetum

pd.set_option('display.max_colwidth', None)

## Instructions

* This script pulls from an atom numbers spreadsheet as well as a directory with folders for each ligand that contain all of the log files for each. 

* See example atom numbers spreadsheet and filesystem for more details  

* Input is only required in Part 2 and Part 3. All other cells can be run without user input. 

# Determine What Ligands Will Be Parameterized and Where to Save Data

In [2]:
# Input either 'All Ligands' or 'Specific Ligands' 
pull_parameters_for = 'Specific Ligands' 

# If 'Specific Ligands', input a list of ligand IDs
Specific_Ligand_IDs = ['10', '12', '14', '16']

# Populate variable excel_name with the desired name for the parameters file (e.g. "myparameters.xlsx") 
# and filepath  (e.g. "~/Desktop/Parameters/") 
# excel_name and excel_filepath will be called in section 13 when exporting parameters  
excel_name = 'Example_Parameters.xlsx'
excel_filepath = 'Parameter_spreadsheets/'

# check if excel_filepath exists and create the directory if it doesn't
isExist = os.path.exists(excel_filepath) #(returns True if exists, False if not)
if not isExist:
    # (create new directory bc it does not exist)
    os.makedirs(excel_filepath)
    print("new directory created ✓")
else:
    print("excel_filepath exists ✓")

excel_filepath exists ✓


# Load Atom Numbers and log filenames and filepaths

This notebook is set up to pull parameters for bisphosphine ligands that have been optimized with PdCl2 and single-point calculations have been run on the optimized structure with and without palladium. 

Then create an excel spreadsheet that contains the names of the log files and the atom numbers both with and without Palladium. These will be read in as Pd_anum_df and NoPd_anum_df.

In [3]:
# New path for ligand folders
Ligands_path = 'DFT_files_atom_nums/Ligand_Calcs/'

# Filepaths for atom numbers spreadsheets
Pd_path = 'DFT_files_atom_nums/SPE_Pd/' #File path for single-point .log files with PdCl2
NoPd_path = 'DFT_files_atom_nums/NoPd/' #File path for single-point .log files without PdCl2
OPT_path = 'DFT_files_atom_nums/OPT/'   #File path for single-point .log files of geometry optimizations


# load in atom numbers spreadsheet
num_columns_Pd = 17   #input the number of columns in the Pd atom number spreadsheet (should not change)
num_columns_NoPd = 15    #input the number of columns in the NoPd atom number spreadsheet (should not change)
Pd_anum_df = pd.read_excel('DFT_files_atom_nums/Atom_numbers_5_2022.xlsx', sheet_name="Pd", usecols=list(range(0,(num_columns_Pd))))
NoPd_anum_df = pd.read_excel('DFT_files_atom_nums/Atom_numbers_5_2022.xlsx', sheet_name="No_Pd", usecols=list(range(0,(num_columns_NoPd))))
# dictionary: key = Ligand ID, value = Ligand Name
ligname_dict = dict(zip(list(Pd_anum_df['Ligand ID']), list(Pd_anum_df.Ligand)))
Pd_anum_df.head()

,Ligand ID,Ligand,P1,P2,Pd,Cl1,Cl2,R1,R2,R3,R4,RBack1,RBack2,Symmetry,Renumber,Chiral,log_name
0,8,RR_DIOP,1,2,70,68,69,12,18,24,30,5,8,C2,False,True,RR_DIOP_1_SPE
1,10,SS_Et_Duphos,2,3,63,62,1,6,8,10,12,4,5,C2,False,True,SS_Et_DuPhos_5_SPE
2,12,W001,60,20,96,73,72,64,88,29,40,79,19,C1,False,True,W001_1_6_F_1_SPE
3,14,M003-1,2,3,54,53,52,20,14,26,32,4,9,C2,False,True,M003_1_5_SPE
4,16,W005-1,60,20,90,70,69,64,85,29,40,76,19,C1,False,True,W005_1_1_SPE


# Generate dictionary with all log names for all conformers



NO USER INPUT REQUIRED BEYOND THIS CELL

In [4]:
# Master_Conf_Dict
# key = ligand id (str)
# value = sub dictionary
# sub dictionary:
# key = conformer number (int)
# value = [opt_name, SPE_name, SPE_NoPd_name]
Master_Conf_Dict = {}

# Iterate through ligand folders
ligand_folders = os.listdir(Ligands_path)

# removes items in filenames in the Ligands_path directory that are not folders (e.g. hidden files like .DS_Store)
for item in ligand_folders:
    try:
        os.listdir(Ligands_path + item)
    except:
        ligand_folders.remove(item)

# Takes each folder name and populates with the ligand ID (string element of folder name before '_') and 
# the names of all of the log folders that should be in there. This is done by recognizing the base name (i.e. Segphos) 
# and the number of coformers, and generating 'Segphos_#', 'Segphos_#_SPE', and 'Segphos_#_SPE_NoPd' for each conformer
# It is critical that each folder is named '<Ligand ID>_<Ligand Name>' and 
# each log file is named <ligand name>_<conformer number>_<.log, _SPE.log, _SPE_NoPd.log for OPT and single points repsectively>.
for folder in ligand_folders:
    ID_i = int(folder.split('_')[0]) # Ligand ID
    sub_dict = {} 
    conformers = [] # will populate with conformer names (e.g. Segphos_1, Segphos_2, etc.)
    
    # find unique conformer names
    for file in os.listdir(Ligands_path + folder):
        # appends only the optimization files to the conformers list (e.g. Segphos_1, Segphos_2, etc.)  
        if re.search('_[12345].log', file) and file[:-4] not in conformers:  # up to 5 conformers as written  
            conformers.append(file[:-4])

    # add conformer number (e.g. 1) and conformer log names 
    # (e.g. 'Segphos_1.log', 'Segphos_1_SPE.log', and 'Segphos_1_SPE_NoPd.log')
    # note that the base name is defined by the optimization log file of conformer 1.
    # This means if conformer 1 is named 'Segphos_1.log', the base name will be Segphos.
    # However, if the base name has something in it like 'tight', that will be included in the base name for every   
    # conformer 
    for conf_num in range(1,len(conformers)+1): # not zero-indexed
        conf_name = conformers[0][:-2] + '_' + str(conf_num) # name of conformer (e.g. 'Segphos_1') 
        conf_opt, conf_SPE, conf_NoPd = conf_name, conf_name + '_SPE', conf_name + '_SPE_NoPd'
        sub_dict[conf_num] = [conf_opt, conf_SPE, conf_NoPd]
    Master_Conf_Dict[ID_i] = sub_dict

# Sort Master_Conf_Dict in order of ascending ligand IDs
Master_Conf_Dict = dict(sorted(Master_Conf_Dict.items()))

## Clean-Up Dictionary with All log Names

* Removes ligands from Master_Conf_Dict if any of their respective log names are not real files

(e.g. if 'Segphos_3.log' is not a real file and the optimization for comformer 3 is actually named 'Segphos_3_tight.log' then an alert will be raised and all coformers of Segphos will be removed from the dictionary)

In [5]:
Problem_Ligands_Dict = {} # Problem ligands: key = ligand_ID, value = list of non-existant ligand names
        
# iterate through all of the ligand folders
for lig_folder in ligand_folders:
    problem_files = []
    
    # record the ligand ID
    lig_ID = int(lig_folder.split('_')[0] ) 
    
    # make a list of all the files in the given folder
    actual_files = os.listdir(Ligands_path + lig_folder)
    
    # make a list of dictionary files (OPTs, SPEs, and NoPd_SPEs)
    dict_files = [file + '.log' for sublist in Master_Conf_Dict[lig_ID].values() for file in sublist]
    

    # check if every file in the dictionary list is in the actual file list
    for file in dict_files:
        if file not in actual_files:
            problem_files.append(file)
    
    # updates Problem_Ligands_Dict with ligands that have naming issues
    if len(problem_files) > 0:
        Problem_Ligands_Dict[lig_ID] = problem_files 
        
# print out the ligands that have issues
for ligand, files in Problem_Ligands_Dict.items():
    print('--------------------------------')
    # f string instead of str.format()
    print(f"Problem with ligand {ligand}:")
    for file in files:
        print(file)
    print('\n\n')
    
# Remove ligands with naming issues from Master_Conf_Dict
for i in Problem_Ligands_Dict:
    del Master_Conf_Dict[i]      

## Check that each folder has the correct ligand ID

* This cell takes the ligand ID from Master_Conf_Dict and makes sure that the SPE, NoPd, and OPT lognames for the lowest energy conformer and recorded in the atom numbers spreadsheet actually exists in that folder

In [6]:
folder_ids = [folder.split('_')[0] for folder in ligand_folders]
folder_dict = dict(zip(folder_ids, ligand_folders))

folders_with_issues = []
for ID, conf_dict_i in Master_Conf_Dict.items():
    ID = str(ID)
    folder_name_i = folder_dict[ID]
    correct_SPE = list(Pd_anum_df[Pd_anum_df['Ligand ID'] == int(ID)]['log_name'])[0] + '.log'
    correct_NoPd_SPE = list(NoPd_anum_df[NoPd_anum_df['Ligand ID'] == int(ID)]['log_name'])[0] + '.log'
    correct_OPT = list(NoPd_anum_df[NoPd_anum_df['Ligand ID'] == int(ID)]['opt_log_name'])[0] + '.log'
    correct_LOGs = [correct_NoPd_SPE, correct_OPT, correct_SPE]
    
    try:
        real_files = os.listdir(Ligands_path + folder_name_i)
        for correct_filename in correct_LOGs:
            if correct_filename not in real_files:
                if folder_name_i not in folders_with_issues:
                    folders_with_issues.append(folder_name_i)
                print(f"{ID} {folder_name_i}.\n{correct_filename} does not exist in the folder.\n")
    except:
        print(f"Problem with {folder_name}. This folder does not exist\n\n\n")
        

print(f"{len(folders_with_issues)} folders have naming inconsistencies")
print(folders_with_issues)


0 folders have naming inconsistencies
[]


## Creates an atom number dataframe with a row for every conformer of every ligand

In [7]:
# Ligand_Filepath_Dict key = ID (int) Value = Filepath pointing to ligand log and com files.
Ligand_Filepath_Dict = {}
for folder in ligand_folders:
    ID = int(folder.split('_')[0])
    filepath = Ligands_path + folder + '/'
    Ligand_Filepath_Dict[ID] = filepath
Ligand_Filepath_Dict = dict(sorted(Ligand_Filepath_Dict.items()))
    

# Create dataframes Conf_Pd_anum_df and Conf_NoPd_anum_df (w/ and w/o PdCl2) 
# with a row for each ligand conformer. An additional row is added for conformer number (1-5)
Conf_Pd_anum_df = pd.DataFrame(columns=list(Pd_anum_df.columns) + ['Conf_Number'])
Conf_NoPd_anum_df = pd.DataFrame(columns=list(NoPd_anum_df.columns) + ['Conf_Number'])


for ID_i, conformers in Master_Conf_Dict.items():
    for conf in conformers:
        # get the row values for the Pd_anum_df (i.e. Ligand ID, Ligand name, atom numbers)
        Pd_anum_row = Pd_anum_df[Pd_anum_df['Ligand ID'] == int(ID_i)].values[0].tolist()
        NoPd_anum_row = NoPd_anum_df[Pd_anum_df['Ligand ID'] == int(ID_i)].values[0].tolist()
        # omit the log name of the lowest energy conformer that is stored in Pd_anum_df
        Pd_anum_row = Pd_anum_row[:-1]
        # omit the lowest energy conformer NoPd_SPE and OPT log names that are stored in Pd_anum_df
        NoPd_anum_row = NoPd_anum_row[:-2]
        
        # update the row from the atom_numbers_spreadsheet to include the the log names for each conformer 
        # as well as the conformer numbers
        [conf_OPT, conf_SPE, conf_NoPd] = Master_Conf_Dict[ID_i][conf]
        #Add filepath info
        filepath = Ligand_Filepath_Dict[ID_i]
        conf_OPT, conf_SPE, conf_NoPd = filepath + conf_OPT, filepath + conf_SPE, filepath + conf_NoPd
        Pd_anum_row = Pd_anum_row + [conf_SPE, conf]
        NoPd_anum_row = NoPd_anum_row + [conf_NoPd, conf_OPT, conf]

        # add the lists as new rows to both the Conf_Pd_anum_df and Conf_NoPd_anum_df
        Conf_Pd_anum_df.loc[len(Conf_Pd_anum_df.index)] = Pd_anum_row
        Conf_NoPd_anum_df.loc[len(Conf_NoPd_anum_df.index)] = NoPd_anum_row

# Updates Ligand ID and conformer number to strings
Conf_Pd_anum_df = Conf_Pd_anum_df.astype({'Ligand ID': str, 'Conf_Number': str})
Conf_NoPd_anum_df = Conf_NoPd_anum_df.astype({'Ligand ID': str, 'Conf_Number': str})

# Updates the Conf_Pd_anum_df and Conf_NoPd_anum_df so that the ligand ID is updated 
# to include the conformer (i.e. for ligand 8 that has two conformers there is a row '8:1' and '8:2') 
Conf_Pd_anum_df['Ligand ID'] = Conf_Pd_anum_df['Ligand ID'] + ':' + Conf_Pd_anum_df['Conf_Number']
Conf_NoPd_anum_df['Ligand ID'] = Conf_NoPd_anum_df['Ligand ID'] + ':' + Conf_NoPd_anum_df['Conf_Number']


display(Conf_Pd_anum_df, Conf_NoPd_anum_df)


,Ligand ID,Ligand,P1,P2,Pd,Cl1,Cl2,R1,R2,R3,R4,RBack1,RBack2,Symmetry,Renumber,Chiral,log_name,Conf_Number
0,8:1,RR_DIOP,1,2,70,68,69,12,18,24,30,5,8,C2,False,True,DFT_files_atom_nums/Ligand_Calcs/8_RR_DIOP/RR_DIOP_1_SPE,1
1,8:2,RR_DIOP,1,2,70,68,69,12,18,24,30,5,8,C2,False,True,DFT_files_atom_nums/Ligand_Calcs/8_RR_DIOP/RR_DIOP_2_SPE,2
2,10:1,SS_Et_Duphos,2,3,63,62,1,6,8,10,12,4,5,C2,False,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_1_SPE,1
3,10:2,SS_Et_Duphos,2,3,63,62,1,6,8,10,12,4,5,C2,False,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_2_SPE,2
4,10:3,SS_Et_Duphos,2,3,63,62,1,6,8,10,12,4,5,C2,False,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_3_SPE,3
5,10:4,SS_Et_Duphos,2,3,63,62,1,6,8,10,12,4,5,C2,False,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_4_SPE,4
6,10:5,SS_Et_Duphos,2,3,63,62,1,6,8,10,12,4,5,C2,False,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_5_SPE,5
7,12:1,W001,60,20,96,73,72,64,88,29,40,79,19,C1,False,True,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_1_SPE,1
8,12:2,W001,60,20,96,73,72,64,88,29,40,79,19,C1,False,True,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_2_SPE,2
9,12:3,W001,60,20,96,73,72,64,88,29,40,79,19,C1,False,True,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_3_SPE,3


,Ligand ID,Ligand,P1,P2,R1,R2,R3,R4,RBack1,RBack2,Symmetry,Renumber,Chiral,log_name,opt_log_name,Conf_Number
0,8:1,RR_DIOP,1,2,12,18,24,30,5,8,C2,False,True,DFT_files_atom_nums/Ligand_Calcs/8_RR_DIOP/RR_DIOP_1_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/8_RR_DIOP/RR_DIOP_1,1
1,8:2,RR_DIOP,1,2,12,18,24,30,5,8,C2,False,True,DFT_files_atom_nums/Ligand_Calcs/8_RR_DIOP/RR_DIOP_2_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/8_RR_DIOP/RR_DIOP_2,2
2,10:1,SS_Et_Duphos,1,2,5,7,9,11,3,4,C2,False,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_1_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_1,1
3,10:2,SS_Et_Duphos,1,2,5,7,9,11,3,4,C2,False,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_2_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_2,2
4,10:3,SS_Et_Duphos,1,2,5,7,9,11,3,4,C2,False,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_3_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_3,3
5,10:4,SS_Et_Duphos,1,2,5,7,9,11,3,4,C2,False,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_4_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_4,4
6,10:5,SS_Et_Duphos,1,2,5,7,9,11,3,4,C2,False,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_5_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_5,5
7,12:1,W001,60,20,64,86,29,40,77,19,C1,False,True,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_1_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_1,1
8,12:2,W001,60,20,64,86,29,40,77,19,C1,False,True,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_2_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_2,2
9,12:3,W001,60,20,64,86,29,40,77,19,C1,False,True,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_3_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_3,3


# Define Functions For Pulling Parameters

## Define Functions to Pull Burried Volume

In [8]:
#This function iterates over a dataframe with the following columns:
#1. Ligand ID, 2. Ligand, 3-13 are atom numbers, and 14 is the name of the log file (without .log) 
def Vbur_one_radius(radius, dataframe, plot_images=False, export_parameters=True):
    bv_dataframe = pd.DataFrame(columns=['Ligand ID', 'Ligand','Vbur%', 'NE', 'NW', 'SE', 'SW'])
    for index, row in dataframe.iterrows():
        enantiomer = False
        if str(row['Ligand ID'])[-2:] == '_1':
            enantiomer = True
        try:
            log_file = row['log_name']
            streams, errors = geetum.get_outstreams(log_file)
            log_coordinates = geetum.get_geom(streams)
            elements = np.array([log_coordinates[i][0] for i in range(len(log_coordinates))])
            coordinates = np.array([np.array(log_coordinates[i][1:]) for i in range(len(log_coordinates))])
            metal_anum = row['Pd']          #Pd atom number (1-indexed)
            
            bv = BuriedVolume(elements, coordinates, metal_anum,
                              excluded_atoms=[row['Cl1'], row['Cl2']],  #Chloride atom numbers (1-indexed)
                              z_axis_atoms=[row['P1'],row['P2']],       #Phosphine atom numbers (1-indexed)
                              xz_plane_atoms=[row['P2']],               #Phosphine oriented east
                              include_hs=True,
                              radius=radius)            
            
            bv.octant_analysis()  
            bv_quads = bv.quadrants['percent_buried_volume']
            bv_octs = bv.octants['percent_buried_volume']
            if enantiomer == False:
                row_i = {'Ligand ID': row['Ligand ID'],'Ligand': row['Ligand'],'Vbur%': 100*bv.fraction_buried_volume,
                         'NE': bv_quads[1], 'NW': bv_quads[2], 'SW':bv_quads[3], 'SE': bv_quads[4],
                        'NE_octant': bv_octs[0], 'NW_octant': bv_octs[1], 'SW_octant': bv_octs[2], 'SE_octant': bv_octs[3]}
            if enantiomer == True:
                row_i = {'Ligand ID': row['Ligand ID'],'Ligand': row['Ligand'],'Vbur%': 100*bv.fraction_buried_volume,
                         'SE': bv_quads[1], 'SW': bv_quads[2], 'NW':bv_quads[3], 'NE': bv_quads[4],
                        'SE_octant': bv_octs[0], 'SW_octant': bv_octs[1], 'NW_octant': bv_octs[2], 'NE_octant': bv_octs[3]}
            if export_parameters==True:
                bv_dataframe = pd.concat([bv_dataframe, pd.DataFrame(row_i, index=[0])], axis=0, ignore_index=True)

            if plot_images==True:
                print('Steric map for ', row['Ligand'], ":")
                bv.plot_steric_map()
        except:
            print('Unable to acquire Vbur parameters for:', row['Ligand'])
    if export_parameters==True:
        name_addon = "_" + str(radius) + '_Ang'
        # rename df columns to be indicative of radius used in buried volume calculation
        bv_dataframe.rename(columns={'Vbur%': 'Vbur%'+ name_addon, 'NE': 'NE' + name_addon, 'NW': 'NW' + name_addon,
                                     'SE': 'SE'+ name_addon, 'SW':'SW' + name_addon, 'NE_octant': 'NE_octant' + name_addon,
                                     'NW_octant': 'NW_octant' + name_addon, 'SW_octant': 'SW_octant' + name_addon,
                                    'SE_octant': 'SE_octant' + name_addon,}, inplace=True)
        return(bv_dataframe)


# This function generates %Vbur (total and quadrants) at defined starting and stopping radius and stepsize
# (e.g. from 2.0Å to 4.0Å at 0.5 Å steps) and returns all of them in a single dataframe.
def get_Vbur(start_r, end_r, step_size, dataframe, plot_images=False, enantiomer=False):
    num_steps = int((end_r-start_r)/step_size + 1)
    radii = [start_r + step_size*i for i in range(num_steps)]
    df_lst = []
    
    for radius in radii:
        df_lst.append(Vbur_one_radius(radius, dataframe, plot_images))
    dfmaster = pd.merge(df_lst[0], df_lst[1])
    
    for i in range(2,len(df_lst)):
        dfmaster = pd.merge(dfmaster, df_lst[i])
    
    return(dfmaster)

def plot_quadrants(radius, dataframe):
    Vbur_one_radius(radius, dataframe, plot_images=True, export_parameters=False)

#test_vbur = get_Vbur(2, 7, 1, Pd_anum_df[:10]) 
#test_vbur.head(5)
#writer = pd.ExcelWriter('Parameter_spreadsheets/' + 'Vbur_2.xlsx', engine='xlsxwriter')
#test_vbur.to_excel(writer, sheet_name= 'Vbur')
#writer.save()

## Define Electronics and Bond-Angle Functions

In [9]:
# This function creates two dataframes with all of the parameters that are pulled using geetum
# one dataframe uses the single-point calculation with PdCl2 and the other without
def bond_vals(dataframe_Pd, dataframe_NoPd):
    bondvals_dataframe_Pd = pd.DataFrame(columns=['Ligand ID', 'Ligand', 'SPE_HF_E', 'Symmetry'])
    bondvals_dataframe_NoPd = pd.DataFrame(columns=['Ligand ID'])
    
    # Iterates through the atom numbers dataframe (with PdCl2 sheet) and pulls parameters based on the atom numbers
    # Parameters are first pulled to a dictionary and then that dictionary is added to the end of the Bondvals_dataframe_Pd
    for index, row in dataframe_Pd.iterrows():
        try:
            str_row = row.astype(str)
            atom_num_dict, SPE_dict, measurements_dict = geetum.get_SPE_Pd(str_row['log_name'], [str_row['P1'], str_row['P2']], [str_row['Pd'], str_row['Cl1'], str_row['Cl2']], [str_row['R1'], str_row['R2'], str_row['R3'], str_row['R4']], [str_row['RBack1'], str_row['RBack2']])            
            row_head = {'Ligand ID': row['Ligand ID'], 'Ligand': row['Ligand'], 'Symmetry': row['Symmetry']}
            row_i = {**row_head,**SPE_dict, **measurements_dict}
            bondvals_dataframe_Pd = pd.concat([bondvals_dataframe_Pd, pd.DataFrame(row_i, index=[0])], axis=0, ignore_index=True)
        except:
            print('Unable to acquire bond/angle parameters (w/ PdCl2) for:', row['Ligand'])
            
    # Iterates through the atom numbers dataframe (w/o PdCl2 sheet) and pulls parameters based on the atom numbers
    # Parameters are first pulled to a dictionary and then that dictionary is added to the end of the Bondvals_dataframe_NoPd
    for index, row in dataframe_NoPd.iterrows():
        try:
            str_row = row.astype(str)
            SPE_NoPd_dict = geetum.get_SPE_NoPd(str_row['log_name'], [str_row['P1'], str_row['P2']], [str_row['R1'], str_row['R2'], str_row['R3'], str_row['R4']], [str_row['RBack1'], str_row['RBack2']])
            polarizability_dict = geetum.get_polarizability(str_row['opt_log_name'])
            row_head = {'Ligand ID': row['Ligand ID']}
            row_i = {**row_head,**SPE_NoPd_dict,**polarizability_dict}
            bondvals_dataframe_NoPd = pd.concat([bondvals_dataframe_NoPd, pd.DataFrame(row_i, index=[0])], axis=0, ignore_index=True)
        except:
            print('Unable to acquire bond/angle parameters (w/o PdCl2) for:', row['Ligand'])
    
    bondvals_dataframe_Pd['Ligand ID'] = bondvals_dataframe_Pd['Ligand ID'].astype(str)
    bondvals_dataframe_NoPd['Ligand ID'] = bondvals_dataframe_NoPd['Ligand ID'].astype(str)
    bondvals_dataframe = pd.merge(bondvals_dataframe_Pd, bondvals_dataframe_NoPd, on='Ligand ID')
    return(bondvals_dataframe)

##Below code is for testing this cell 

#test_bondvals = bond_vals(Conf_Pd_anum_df2, Conf_NoPd_anum_df2)
#test_bondvals.head(20)


# Renumber Atoms Based On X/P–Cback Occupancy

In [10]:
if pull_parameters_for == 'Specific Ligands':
    Conf_Pd_anum_df2 = Conf_Pd_anum_df.loc[Conf_Pd_anum_df.apply(lambda row: row['Ligand ID'].split(':')[0] in Specific_Ligand_IDs, axis=1)]
    Conf_NoPd_anum_df2 = Conf_NoPd_anum_df.loc[Conf_NoPd_anum_df.apply(lambda row: row['Ligand ID'].split(':')[0] in Specific_Ligand_IDs, axis=1)]

if pull_parameters_for == 'All_Ligands':
    Conf_Pd_anum_df2, Conf_NoPd_anum_df2 = Conf_Pd_anum_df, Conf_NoPd_anum_df
##########################################################################################################

Pd_anum_df2 = pd.DataFrame(columns=['Ligand ID', 'Ligand', 'P1', 'P2', 'Pd', 'Cl1', 'Cl2', 'R1', 'R2',
       'R3', 'R4', 'RBack1', 'RBack2', 'Symmetry', 'Chiral', 'log_name'])
NoPd_anum_df2 = pd.DataFrame(columns=['Ligand ID', 'Ligand', 'P1', 'P2', 'R1', 'R2', 'R3', 'R4',
       'RBack1', 'RBack2', 'Symmetry', 'Chiral', 'log_name', 'opt_log_name'])
bond_vals_df0 = bond_vals(Conf_Pd_anum_df2, Conf_NoPd_anum_df2)
bond_vals_df0 = bond_vals_df0.set_index('Ligand ID')
working_ligands = list(bond_vals_df0.index)   # This returns a list of ligand IDs for ligands that work in bond_vals function  

for index, row in Conf_Pd_anum_df2.iterrows():     #Pd block
    lig_ID = str(row['Ligand ID'])
    P1_Rback_occ = bond_vals_df0.loc[lig_ID,'P1_RBack1_bond_occ']
    P2_Rback_occ = bond_vals_df0.loc[lig_ID,'P2_RBack2_bond_occ']
    if lig_ID not in working_ligands:
        print(lig_ID, ' was not added to new atom number list dued to failed parameterization!')
    else:
        if row['Renumber'] == False:
            #Pd_anum block 44
            row_Pd = row.drop('Renumber')
            Pd_anum_df2.loc[len(Pd_anum_df2.index)] = list(row_Pd[:-1])
        else:
            if P2_Rback_occ > P1_Rback_occ:
                #Pd_anum block 
                Ligand_ID = row['Ligand ID']
                Ligand = row['Ligand']
                P1 = row['P2']
                P2 = row['P1']
                Pd = row['Pd']
                Cl1 = row['Cl2']
                Cl2 = row['Cl1']
                R1 = row['R4']
                R2 = row['R3']
                R3 = row['R2']
                R4 = row['R1']
                RBack1 = row['RBack2']
                RBack2 = row['RBack1']
                Symmetry = row['Symmetry']
                Chiral = row['Chiral']
                log_name = row['log_name']
                new_row = [Ligand_ID, Ligand,  P1, P2, Pd, Cl1, Cl2, R1, R2, R3, R4, RBack1, RBack2, Symmetry, Chiral, log_name]
                Pd_anum_df2.loc[len(Pd_anum_df2.index)] = new_row
            else:
                #Pd_anum block  asdf
                row_Pd = row.drop('Renumber')
                Pd_anum_df2.loc[len(Pd_anum_df2.index)] = list(row_Pd[:-1])                
                
                
for index, row in Conf_NoPd_anum_df2.iterrows():     #NoPd block
    lig_ID = str(row['Ligand ID'])
    P1_Rback_occ = bond_vals_df0.loc[lig_ID,'P1_RBack1_bond_occ']
    P2_Rback_occ = bond_vals_df0.loc[lig_ID,'P2_RBack2_bond_occ']
    if lig_ID not in working_ligands:
        print(lig_ID, ' was not added to new atom number list dued to failed parameterization!')
    else:
        if row['Renumber'] == False:
            row_NoPd = row.drop('Renumber')
            NoPd_anum_df2.loc[len(NoPd_anum_df2.index)] = list(row_NoPd[:-1])
        else:
            if P2_Rback_occ > P1_Rback_occ:
                #NoPd_anum block 
                Ligand_ID = row['Ligand ID']
                Ligand = row['Ligand']
                P1 = row['P2']
                P2 = row['P1']
                R1 = row['R4']
                R2 = row['R3']
                R3 = row['R2']
                R4 = row['R1']
                RBack1 = row['RBack2']
                RBack2 = row['RBack1']
                Symmetry = row['Symmetry']
                Chiral = row['Chiral']
                log_name = row['log_name']
                opt_log_name = row['opt_log_name']
                new_row = [Ligand_ID, Ligand, P1, P2, R1, R2, R3, R4, RBack1, RBack2, Symmetry,  Chiral, log_name, opt_log_name]
                NoPd_anum_df2.loc[len(NoPd_anum_df2.index)] = new_row
            else:
                #NoPd_anum block 123
                row_NoPd = row.drop('Renumber')
                NoPd_anum_df2.loc[len(NoPd_anum_df2.index)] = list(row_NoPd[:-1])
    

                
display(Pd_anum_df2, NoPd_anum_df2)

,Ligand ID,Ligand,P1,P2,Pd,Cl1,Cl2,R1,R2,R3,R4,RBack1,RBack2,Symmetry,Chiral,log_name
0,10:1,SS_Et_Duphos,2,3,63,62,1,6,8,10,12,4,5,C2,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_1_SPE
1,10:2,SS_Et_Duphos,2,3,63,62,1,6,8,10,12,4,5,C2,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_2_SPE
2,10:3,SS_Et_Duphos,2,3,63,62,1,6,8,10,12,4,5,C2,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_3_SPE
3,10:4,SS_Et_Duphos,2,3,63,62,1,6,8,10,12,4,5,C2,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_4_SPE
4,10:5,SS_Et_Duphos,2,3,63,62,1,6,8,10,12,4,5,C2,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_5_SPE
5,12:1,W001,60,20,96,73,72,64,88,29,40,79,19,C1,True,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_1_SPE
6,12:2,W001,60,20,96,73,72,64,88,29,40,79,19,C1,True,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_2_SPE
7,12:3,W001,60,20,96,73,72,64,88,29,40,79,19,C1,True,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_3_SPE
8,14:1,M003-1,2,3,54,53,52,20,14,26,32,4,9,C2,True,DFT_files_atom_nums/Ligand_Calcs/14_M003_1/M003_1_1_SPE
9,14:2,M003-1,2,3,54,53,52,20,14,26,32,4,9,C2,True,DFT_files_atom_nums/Ligand_Calcs/14_M003_1/M003_1_2_SPE


,Ligand ID,Ligand,P1,P2,R1,R2,R3,R4,RBack1,RBack2,Symmetry,Chiral,log_name,opt_log_name
0,10:1,SS_Et_Duphos,1,2,5,7,9,11,3,4,C2,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_1_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_1
1,10:2,SS_Et_Duphos,1,2,5,7,9,11,3,4,C2,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_2_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_2
2,10:3,SS_Et_Duphos,1,2,5,7,9,11,3,4,C2,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_3_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_3
3,10:4,SS_Et_Duphos,1,2,5,7,9,11,3,4,C2,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_4_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_4
4,10:5,SS_Et_Duphos,1,2,5,7,9,11,3,4,C2,True,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_5_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/10_SS_Et_DuPhos/SS_Et_DuPhos_5
5,12:1,W001,60,20,64,86,29,40,77,19,C1,True,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_1_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_1
6,12:2,W001,60,20,64,86,29,40,77,19,C1,True,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_2_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_2
7,12:3,W001,60,20,64,86,29,40,77,19,C1,True,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_3_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/12_W001_1_6_F/W001_1_6_F_3
8,14:1,M003-1,2,3,20,14,26,32,4,9,C2,True,DFT_files_atom_nums/Ligand_Calcs/14_M003_1/M003_1_1_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/14_M003_1/M003_1_1
9,14:2,M003-1,2,3,20,14,26,32,4,9,C2,True,DFT_files_atom_nums/Ligand_Calcs/14_M003_1/M003_1_2_SPE_NoPd,DFT_files_atom_nums/Ligand_Calcs/14_M003_1/M003_1_2


## Check that every conformer for each ligand has the same P1/P2 designation

In [11]:
# Generate a list of all the ligand IDs for which parameters were pulled.
IDs_Pulled = []
for i in Pd_anum_df2['Ligand ID'].to_list():
    ID = i.split(':')[0]
    if ID not in IDs_Pulled:
        IDs_Pulled.append(ID)
        
# Check that the atom numbers are the same for every row of a given ID
inconsistent_P1_P2_ligands = [] 
for ID in IDs_Pulled:
    # Select all conformers for a given ID
    Sub_df_Pd = Pd_anum_df2.loc[Pd_anum_df2.apply(lambda row: ID == row['Ligand ID'].split(':')[0], axis = 1)]
    Sub_df_NoPd = NoPd_anum_df2.loc[NoPd_anum_df2.apply(lambda row: ID == row['Ligand ID'].split(':')[0], axis = 1)]
    
    # make a list of atom number columns for Pd and NoPd dataframes
    Pd_atom_number_columns, NoPd_atom_number_columns = Pd_anum_df2.columns[2:-3], NoPd_anum_df2.columns[2:-4]
    
    # Check that the atom numbers in each column are the same
    unique_anumbers_Pd = [len(pd.unique(Sub_df_Pd[atom])) for atom in Pd_atom_number_columns]
    unique_anumbers_NoPd = [len(pd.unique(Sub_df_NoPd[atom])) for atom in NoPd_atom_number_columns]
    
    inconsistent_Pd = True in (unique_anumb > 1 for unique_anumb in unique_anumbers_Pd)
    inconsistent_NoPd = True in (unique_anumb > 1 for unique_anumb in unique_anumbers_NoPd)
    
    # Print warning that atom numbers are inconsistent across conformers for a given ligand
    if inconsistent_Pd or inconsistent_NoPd:
        inconsistent_P1_P2_ligands.append(ID)

print('P1/P2 convention is inconsistent for the following ligands:\n')
for ID in inconsistent_P1_P2_ligands:
    print(f"{ligname_dict[int(ID)]} (ID {ID})")
      
    

P1/P2 convention is inconsistent for the following ligands:



# Generate Atom Number Spreadsheets w/ Enantiomers

In [12]:
# Renumbering is not required for Vbur parameters. Instead we simply switch the lables on North and South octants
# and quadrants (e.g. NW_4Å becomes SW_4Å)

# Pd_anum_df3 and NoPd_anum_df3 just have extra rows with the same atom numbers
# Pd_anum_df4 and NoPd_anum_df4 have updated atom numbers where R2/3 and R3/4 are switched.

Pd_anum_df3 = pd.DataFrame(columns=['Ligand ID', 'Ligand', 'P1', 'P2', 'Pd', 'Cl1', 'Cl2', 'R1', 'R2',
       'R3', 'R4', 'RBack1', 'RBack2', 'Symmetry', 'log_name'])
NoPd_anum_df3 = pd.DataFrame(columns=['Ligand ID', 'Ligand', 'P1', 'P2', 'R1', 'R2', 'R3', 'R4',
       'RBack1', 'RBack2', 'Symmetry', 'log_name',  'opt_log_name'])
Pd_anum_df4 = pd.DataFrame(columns=['Ligand ID', 'Ligand', 'P1', 'P2', 'Pd', 'Cl1', 'Cl2', 'R1', 'R2',
       'R3', 'R4', 'RBack1', 'RBack2', 'Symmetry', 'log_name'])
NoPd_anum_df4 = pd.DataFrame(columns=['Ligand ID', 'Ligand', 'P1', 'P2', 'R1', 'R2', 'R3', 'R4',
       'RBack1', 'RBack2','Symmetry', 'log_name', 'opt_log_name'])

##### Pd_anum_df3 and NoPd_anum_df3 #####
# iterates through Pd_anum_df2 and adds rows for both base ligand and enantiomer (e.g. R_Segphos and R_Segphos_ENANT)
# Pd block 
for index, row in Pd_anum_df2.iterrows():   
    if row['Chiral'] == True:
        new_row = list(row)
        del new_row[-2] # Removes Chiral = T/F column from the row.
        Pd_anum_df3.loc[len(Pd_anum_df3.index)] = new_row #Adds row for calculated enantiomer
        new_row[0] = str(new_row[0]) + '_1'
        new_row[1] = new_row[1] + '_ENANT'
        Pd_anum_df3.loc[len(Pd_anum_df3.index)] = new_row #Adds row for new enantiomer (with same atom numbers for now)
    else:
        new_row = list(row)
        del new_row[-2]
        Pd_anum_df3.loc[len(Pd_anum_df3.index)] = new_row
        
# NoPd block         
for index, row in NoPd_anum_df2.iterrows():   
    if row['Chiral'] == True:
        new_row = list(row)
        del new_row[-3]
        NoPd_anum_df3.loc[len(NoPd_anum_df3.index)] = new_row #Adds row for calculated enantiomer
        new_row[0] = str(new_row[0]) + '_1'
        new_row[1] = new_row[1] + '_ENANT'
        NoPd_anum_df3.loc[len(NoPd_anum_df3.index)] = new_row #Adds row for new enantiomer (with same atom numbers for now)
    else:
        new_row = list(row)
        del new_row[-3]
        NoPd_anum_df3.loc[len(NoPd_anum_df3.index)] = new_row

##### Pd_anum_df4 and NoPd_anum_df4 #####
for index, row in Pd_anum_df3.iterrows():  #Pd Block
    if str(row['Ligand ID'])[-2:] == '_1':
        Ligand_ID = row['Ligand ID']
        Ligand = row['Ligand']
        P1 = row['P1']
        P2 = row['P2']
        Pd = row['Pd']
        Cl1 = row['Cl1']
        Cl2 = row['Cl2']
        R1 = row['R2']
        R2 = row['R1']
        R3 = row['R4']
        R4 = row['R3']
        RBack1 = row['RBack1']
        RBack2 = row['RBack2']
        Symmetry = row['Symmetry']
        log_name = row['log_name']
        new_row = [Ligand_ID, Ligand, P1, P2, Pd, Cl1, Cl2, R1, R2, R3, R4, RBack1, RBack2, Symmetry, log_name]
        Pd_anum_df4.loc[len(Pd_anum_df4.index)] = new_row
    else:
        new_row = list(row)
        Pd_anum_df4.loc[len(Pd_anum_df4.index)] = new_row
        
        
for index, row in NoPd_anum_df3.iterrows():  #NoPd Block
    if str(row['Ligand ID'])[-2:] == '_1':
        Ligand_ID = row['Ligand ID']
        Ligand = row['Ligand']
        P1 = row['P1']
        P2 = row['P2']
        R1 = row['R2']
        R2 = row['R1']
        R3 = row['R4']
        R4 = row['R3']
        RBack1 = row['RBack1']
        RBack2 = row['RBack2']
        Symmetry = row['Symmetry']
        log_name = row['log_name']
        opt_log_name = row['opt_log_name']
        new_row = [Ligand_ID, Ligand, P1, P2, R1, R2, R3, R4, RBack1, RBack2, Symmetry, log_name, opt_log_name]
        NoPd_anum_df4.loc[len(NoPd_anum_df4.index)] = new_row
    else:
        new_row = list(row)
        NoPd_anum_df4.loc[len(NoPd_anum_df4.index)] = new_row

# Run Functions to Pull Parameters (Enantiomers)

In [13]:
Vbur_df = get_Vbur(3, 7, 1.0, Pd_anum_df3[:])      # This must be run on Pd_anum_df3
print('\n\nDone Running Vbur collection')

bond_vals_df = bond_vals(Pd_anum_df4[:], NoPd_anum_df4[:])
print('\n\nDone Running bondvals collection')



Done Running Vbur collection


Done Running bondvals collection


## Combine Vbur and BondVals into a single dataframe

In [14]:
del Vbur_df['Ligand']
Vbur_df['Ligand ID'] = Vbur_df['Ligand ID'].astype(str)
df_all_params = pd.merge(bond_vals_df, Vbur_df)

errorcount = 0

# Removes any ligands that have any errors in the parameter acquisition
ligands_with_errors = [] # List of Ligand IDs with Errors
for index, row in df_all_params.iterrows():
    row_lc = [i.lower() for i in row if type(i) == str] # lower case row
    if 'error' in row_lc:
        ID_i = row['Ligand ID'].split(':')[0]
        print(f"Error detected in {row['Ligand ID']}")
        if ID_i not in ligands_with_errors:
            ligands_with_errors.append(ID_i)

# Remove ligands where at least one conformer has an error
for index, row in df_all_params.iterrows():
    ID_i = row['Ligand ID'].split(':')[0]
    if ID_i in ligands_with_errors:
        df_all_params.drop(index, inplace=True)            
        
print('The following ligands were removed because they contain one or more errors:\n')
for ID in ligands_with_errors:
    print(f"{ligname_dict[int(ID)]} ({ID})")
        
df_all_params[df_all_params.columns[4:]]= df_all_params[df_all_params.columns[4:]].astype(float)

# #generate SPE_G: HF energy from single point
df_all_params['SPE_G'] = df_all_params.apply(lambda row: float(row.SPE_HF_E) + float(row.G) - float(row.OPT_HF_E), axis=1)
print('all parameters df:')
df_all_params.head()

The following ligands were removed because they contain one or more errors:

all parameters df:


,Ligand ID,Ligand,SPE_HF_E,Symmetry,R1_NBO,R2_NBO,RBack1_NBO,R3_NBO,R4_NBO,RBack2_NBO,...,Vbur%_7.0_Ang,NE_7.0_Ang,NW_7.0_Ang,SE_7.0_Ang,SW_7.0_Ang,NE_octant_7.0_Ang,NW_octant_7.0_Ang,SW_octant_7.0_Ang,SE_octant_7.0_Ang,SPE_G
0,10:1,SS_Et_Duphos,-2590.0384975,C2,-0.50524,-0.48092,-0.34102,-0.47646,-0.49647,-0.35043,...,33.046302,32.671934,33.425081,32.095262,33.992930,9.716901,2.245208,12.969151,2.629467,-2589.570779
1,10:1_1,SS_Et_Duphos_ENANT,-2590.0384975,C2,-0.48092,-0.50524,-0.34102,-0.49647,-0.47646,-0.35043,...,33.046302,32.095262,33.992930,32.671934,33.425081,2.629467,12.969151,2.245208,9.716901,-2589.570779
2,10:2,SS_Et_Duphos,-2590.0395706,C2,-0.48946,-0.48645,-0.34275,-0.47790,-0.49908,-0.34885,...,33.039542,32.698405,33.209327,31.918218,34.332216,10.345946,3.831200,14.581898,2.780324,-2589.570984
3,10:2_1,SS_Et_Duphos_ENANT,-2590.0395706,C2,-0.48645,-0.48946,-0.34275,-0.49908,-0.47790,-0.34885,...,33.039542,31.918218,34.332216,32.698405,33.209327,2.780324,14.581898,3.831200,10.345946,-2589.570984
4,10:3,SS_Et_Duphos,-2590.0350216,C2,-0.49380,-0.48392,-0.34076,-0.47434,-0.50218,-0.34812,...,33.150692,33.880498,32.047728,32.677342,33.997199,15.475655,3.991734,14.173730,3.434986,-2589.567942


Create Conformer dictionaries

* Generates one dictionary for base ligands and one for enantiomers (Enant_Conf_Dict, Base_Conf_Dict)
* Each key is a ligand ID (without enantiomer info) and each value is a list of the specific IDs for all conformers of the ligand within the all_params dataframe. (includes conformer and enantiomer info, e.g. ['8:1_1', '8:2_1', ..]). 

In [15]:
# All_Base/Enant_Conf: list of all full ids (e.g. '516:1' from the base/enantiomer ligands)
All_Base_Conf = [ID for ID in df_all_params['Ligand ID'] if ID[-2:] != '_1']
All_Enant_Conf = [ID for ID in df_all_params['Ligand ID'] if ID[-2:] == '_1']

# Iterates through both lists and creates Unique_Base_IDs and Unique_Enant_IDs (e.g. '8', instead of '8:1', '8:2', ...)
Unique_Base_IDs = list(set([ID.split(':')[0] for ID in All_Base_Conf]))
Unique_Enant_IDs = list(set([ID.split(':')[0] for ID in All_Enant_Conf])) # do not include '_1' ending

# Create Enant/Base_Conf_Dict:
# Base_Conf_Dict: key = ID (e.g. '8'), value = ID of each conformer (e.g. ['8:1', '8:2', ..])
# Enant_Conf_Dict: key = ID (e.g. '8'), value = ID of each conformer (e.g. ['8:1_1', '8:2_1', ..])
Enant_Conf_Dict, Base_Conf_Dict = {}, {}

# Iterate through Unique_Base_IDs. For ID in Unique_Base_IDs: individual_confs = [] for conf in All_Base_Conf: if ID in conf append id to individual_confs
for ID in Unique_Base_IDs:
    individual_confs = []
    for conf in All_Base_Conf:
        if ID == conf.split(':')[0]:
            individual_confs.append(conf)
    Base_Conf_Dict[ID] = individual_confs
    
for ID in Unique_Enant_IDs:
    individual_confs = []
    for conf in All_Enant_Conf:
        if ID == conf.split(':')[0]:
            individual_confs.append(conf)
    Enant_Conf_Dict[ID] = individual_confs

# Create Dataframes (Raw parameters prior to symmetry adaptation)

* Boltz_All_Params = Boltzmann Weighted Parameters
* Min_All_Params = Minimum Value Parameters
* Max_All_Params = Maximum Value Paramters
* Lowconf_All_Params = Low-Energy Conformer Parameters

In [16]:
# Create a dataframe with new relative energies and Boltzmann values
df_all_params_Boltz = pd.DataFrame(columns=list(df_all_params.columns)+ ['E_Rel', 'Boltz_Weight', 'Boltz_Prob'])

# Create  Dataframes for Boltzmann, Minimum value, Maximum Value, and Low Energy conformer parameters
# named Boltz_All_Params, Min__All_Params, Max_All_Params, Lowconf__All_Params, respectively
Boltz_All_Params = pd.DataFrame(columns= [i for i in list(df_all_params.columns) if i not in ['SPE_HF_E', 'G', 'OPT_HF_E', 'SPE_G', 'E_Rel', 'Boltz_Weight', 'Boltz_Prob']] )
Min_All_Params = pd.DataFrame(columns= [i for i in list(df_all_params.columns) if i not in ['SPE_HF_E', 'G', 'OPT_HF_E', 'SPE_G', 'E_Rel', 'Boltz_Weight', 'Boltz_Prob']] )
Max_All_Params = pd.DataFrame(columns= [i for i in list(df_all_params.columns) if i not in ['SPE_HF_E', 'G', 'OPT_HF_E', 'SPE_G', 'E_Rel', 'Boltz_Weight', 'Boltz_Prob']] )
Lowconf_All_Params = pd.DataFrame(columns= [i for i in list(df_all_params.columns) if i not in ['SPE_HF_E', 'G', 'OPT_HF_E', 'SPE_G', 'E_Rel', 'Boltz_Weight', 'Boltz_Prob']] )


# Populate Boltz_, Min_, Max_, and Lowconf_Params for Base Ligands (i.e. not enantiomers)
temp = 298 # temperature in kelvin that will be used for Boltzmann averaging
for ID, confs in Base_Conf_Dict.items():
    # create a temporary dataframe with all conformers of a given ligand ID
    temp_df = df_all_params[df_all_params['Ligand ID'].isin(confs)]    

    
    # calculate the relative energy within the temporary dataframe
    min_energy = min(list(temp_df.SPE_G))
    temp_df['E_Rel'] = temp_df.apply(lambda row: 627.5*(row.SPE_G - min_energy), axis=1)
    temp_df['Boltz_Weight'] = temp_df.apply(lambda row: np.exp(-row.E_Rel/(.001987*temp)), axis=1)
    temp_df['Boltz_Prob'] = temp_df.apply(lambda row: row.Boltz_Weight/sum(list(temp_df.Boltz_Weight)) , axis=1)
    
    # Add new parameters with E_Rel, Boltz_weight, and Boltz_Prob to df_all_params_Boltz
    df_all_params_Boltz = pd.concat([df_all_params_Boltz, temp_df], axis=0, ignore_index=True)
    
    # create four dictionaries: min_dict, max_dict, boltz_dict, and lowconf_dict
    # they will contain the minimum, maximum, boltzman weighted, and low conformer values
    # and will have the followign key: value pairs
    # {'Ligand ID': 8, 'Ligand': 'DIOP', 'Symmetry': 'C2', 'P1_NMR': '254.1', ...}
    non_param_dict = {'Ligand ID': ID, 'Ligand': ligname_dict[int(ID)], 'Symmetry': temp_df.Symmetry.iloc[0]}
    min_dict, max_dict, boltz_dict, lowconf_dict = non_param_dict.copy(), non_param_dict.copy(), non_param_dict.copy(), non_param_dict.copy()
    for param in temp_df.columns:
        if param not in ['Ligand ID', 'Ligand', 'SPE_HF_E', 'Symmetry', 'G', 'OPT_HF_E', 'SPE_G', 'E_Rel', 'Boltz_Weight', 'Boltz_Prob']:
            min_i, max_i = min(list(temp_df[param])), max(list(temp_df[param]))
            # This try/except statement is to avoid errors when two conformers have exactly the same relative
            # energy, E = 0. We average the degenerate conformer values.
            try:
                lowconf_i = float(temp_df[temp_df.E_Rel == float(0)][param])
            except:
                lowconf_i = float(temp_df[temp_df.E_Rel == float(0)][param].mean())
            param_boltz_pairs = list(zip(list(temp_df[param]), list(temp_df.Boltz_Prob)))
            boltz_i = sum([i[0]*i[1] for i in param_boltz_pairs])

            min_dict[param], max_dict[param],  = min_i, max_i 
            boltz_dict[param], lowconf_dict[param] = boltz_i, lowconf_i
            
                        
    # Add min_dict, max_dict, boltz_dict, and low_conf_dict as rows in Boltz_All_Params, Min_All_Params, etc. 
    Boltz_All_Params.loc[len(Boltz_All_Params.index)] = boltz_dict
    Min_All_Params.loc[len(Min_All_Params.index)] = min_dict
    Max_All_Params.loc[len(Max_All_Params.index)] = max_dict
    Lowconf_All_Params.loc[len(Lowconf_All_Params.index)] = lowconf_dict
    
# Populate Boltz_, Min_, Max_, and Lowconf_Params for ENANT Ligands (i.e. not enantiomers)
temp = 298 #temperature in kelvin that will be used for Boltzmann averaging
for ID, confs in Enant_Conf_Dict.items():
    # create a temporary dataframe where ligand ID == ID
    temp_df = df_all_params[df_all_params['Ligand ID'].isin(confs)]
    
    # calculate the relative energy within the temporary dataframe
    min_energy = min(list(temp_df.SPE_G))
    temp_df['E_Rel'] = temp_df.apply(lambda row: 627.5*(row.SPE_G - min_energy), axis=1)
    temp_df['Boltz_Weight'] = temp_df.apply(lambda row: np.exp(-row.E_Rel/(.001987*temp)), axis=1)
    temp_df['Boltz_Prob'] = temp_df.apply(lambda row: row.Boltz_Weight/sum(list(temp_df.Boltz_Weight)) , axis=1)
    
    # Add new parameters with E_Rel, Boltz_weight, and Boltz_Prob to df_all_params_Boltz
    df_all_params_Boltz = pd.concat([df_all_params_Boltz, temp_df], axis=0, ignore_index=True)
    
    # create min_dict, max_dict, boltz_dict, and lowconf_dict
    # dictionaries: {'Ligand ID': 8, 'Ligand': 'DIOP', 'Symmetry': 'C2', 'P1_NMR': '254.1', ...}
    non_param_dict = {'Ligand ID': ID + '_1', 'Ligand': ligname_dict[int(ID)] + '_ENANT', 'Symmetry': temp_df.Symmetry.iloc[0]}
    min_dict, max_dict, boltz_dict, lowconf_dict = non_param_dict.copy(), non_param_dict.copy(), non_param_dict.copy(), non_param_dict.copy()
    for param in temp_df.columns:
        if param not in ['Ligand ID', 'Ligand', 'SPE_HF_E', 'Symmetry', 'G', 'OPT_HF_E', 'SPE_G', 'E_Rel', 'Boltz_Weight', 'Boltz_Prob']:
            min_i, max_i = min(list(temp_df[param])), max(list(temp_df[param]))
            # This try/except statement is to avoid errors when two conformers have exactly the same relative
            # energy, E = 0. We average the degenerate conformer values.
            try:
                lowconf_i = float(temp_df[temp_df.E_Rel == float(0)][param])
            except:
                lowconf_i = float(temp_df[temp_df.E_Rel == float(0)][param].mean())
            param_boltz_pairs = list(zip(list(temp_df[param]), list(temp_df.Boltz_Prob)))
            boltz_i = sum([i[0]*i[1] for i in param_boltz_pairs])

            min_dict[param], max_dict[param] = min_i, max_i 
            boltz_dict[param], lowconf_dict[param] = boltz_i, lowconf_i
            
                        
    # Add min_dict, max_dict, boltz_dict, and low_conf_dict as rows in Boltz_All_Params, Min_All_Params, etc. 
    Boltz_All_Params.loc[len(Boltz_All_Params.index)] = boltz_dict
    Min_All_Params.loc[len(Min_All_Params.index)] = min_dict
    Max_All_Params.loc[len(Max_All_Params.index)] = max_dict
    Lowconf_All_Params.loc[len(Lowconf_All_Params.index)] = lowconf_dict
    
#df_all_params_Boltz.to_excel('Params_With_Weights.xlsx')    
Boltz_All_Params.head()
    

<ipython-input-16-a6459f985314>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['E_Rel'] = temp_df.apply(lambda row: 627.5*(row.SPE_G - min_energy), axis=1)
<ipython-input-16-a6459f985314>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['Boltz_Weight'] = temp_df.apply(lambda row: np.exp(-row.E_Rel/(.001987*temp)), axis=1)
<ipython-input-16-a6459f985314>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,Ligand ID,Ligand,Symmetry,R1_NBO,R2_NBO,RBack1_NBO,R3_NBO,R4_NBO,RBack2_NBO,P1_NMR,...,SE_octant_6.0_Ang,Vbur%_7.0_Ang,NE_7.0_Ang,NW_7.0_Ang,SE_7.0_Ang,SW_7.0_Ang,NE_octant_7.0_Ang,NW_octant_7.0_Ang,SW_octant_7.0_Ang,SE_octant_7.0_Ang
0,14,M003-1,C2,-0.348536,-0.338282,-0.384485,-0.355328,-0.331774,-0.381278,223.712573,...,35.534073,53.592532,55.270883,50.440636,62.830239,45.828369,27.205555,16.975022,14.421459,32.967635
1,16,W005-1,C1,-0.358820,-0.374498,-0.327997,-0.338113,-0.329156,-0.495009,239.507630,...,34.533808,50.129636,48.682426,49.608254,54.009480,48.218381,18.751976,20.899979,14.931458,31.216357
2,12,W001,C1,-0.353206,-0.366268,-0.330529,-0.338378,-0.328816,-0.501790,239.946068,...,35.956625,47.289078,45.189811,48.459084,52.847760,42.659655,11.988756,18.541030,8.619142,33.835847
3,10,SS_Et_Duphos,C2,-0.503588,-0.482743,-0.343348,-0.477474,-0.493849,-0.346986,188.551043,...,4.578423,33.083588,33.096035,33.357484,32.118801,33.762033,9.199751,5.068577,12.892721,2.883285
4,14_1,M003-1_ENANT,C2,-0.338282,-0.348536,-0.384485,-0.331774,-0.355328,-0.381278,223.712573,...,30.500522,53.592532,62.830239,45.828369,55.270883,50.440636,32.967635,14.421459,16.975022,27.205555


# Covert Raw Parameters to Symmetry Adapted Parameters

* sym_adapt_boltz = Boltzmann Weighted Parameters
* sym_adapt_min = Minimum Value Parameters
* sym_adapt_max = Maximum Value Paramters
* sym_adapt_lowconf = Low-Energy Conformer Parameters


NOTE TO SELF: Continue proofreading from here!

In [17]:
#base parameters are the unique parameters w/o symmetry or min/max/avg
#(e.g. base parameters would be P1_NMR or P2_NMR, not X_NMR_min or X_NMR_max)
base_params = sum(geetum.C1_dict.values(), []) 

#Symmetry adapted parameters give min/max/avg for each base parameter
#(e.g. P1_NMR_min, P1_NMR_max, and P1_NMR_avg)
sym_adapt_params = [[i + '_min', i + '_max', i + '_avg'] for i in base_params]
sym_adapt_params = sum(sym_adapt_params, [])


# Define funciton to symmetry adapt a dataframe of parameters
def symmetry_adapt(dataframe):

    #sym_adapt_df is a dataframe that will be populated with the symmetry-adapted parameters for each ligand
    sym_adapt_df = pd.DataFrame(columns=['Ligand ID', 'Ligand']+sym_adapt_params)

    for index, row in dataframe.iterrows():
        if 'Error' in list(row):
            print(f"1+ error values were found in ligand {row['Ligand']} (ID: {row['Ligand ID']})")
            continue
        row_i = {'Ligand ID': row['Ligand ID'], 'Ligand': row['Ligand']}
        for p in geetum.global_parameters: #this keeps the values for parameters not affected by symmetry (HOMO, Pd_LP_E, etc.)
            row_i[p] = row[p]
        if row.Symmetry == 'C2':
            symmetry_dict = geetum.C2_dict
        elif row.Symmetry == 'C2v':
            symmetry_dict = geetum.C2v_dict
        elif row.Symmetry == 'C1':
            symmetry_dict = geetum.C1_dict
        else:
            print(f"Error, incorrect symmmetry designation for {row['Ligand']} (ID: {row['Ligand ID']})")
            continue

        for params in symmetry_dict.values():   #This updates parameters that are affected by symmetry to have min/max/avg vals
            old_vals = [float(row[p]) for p in params] 
            #This gives discrete values for symmetry-equivalent groups 
            for p in params:
                p_min, p_max, p_avg = min(old_vals), max(old_vals), statistics.mean(old_vals)
                row_i[p + '_min'] = p_min
                row_i[p + '_max'] = p_max
                row_i[p + '_avg'] = p_avg
            
        for ratio_i, params in geetum.ratio_dict.items():  #Note: The ratios are optimized for C2 chiral ligands.
            ratio_1 = row[params[0][0]]/row[params[0][1]]
            ratio_2 = row[params[1][0]]/row[params[1][1]]
            ratios = [ratio_1, ratio_2]
            ratio_min, ratio_max, ratio_avg = min(ratios), max(ratios), statistics.mean(ratios)
            row_i[ratio_i + '_min'] = ratio_min
            row_i[ratio_i + '_max'] = ratio_max
            row_i[ratio_i + '_avg'] = ratio_avg
            
        sym_adapt_df = pd.concat([sym_adapt_df, pd.DataFrame(row_i, index=[0])], axis=0, ignore_index=True)


    sym_adapt_df.set_index('Ligand ID', inplace=True)
    return sym_adapt_df


sym_adapt_boltz = symmetry_adapt(Boltz_All_Params)
sym_adapt_min = symmetry_adapt(Min_All_Params)
sym_adapt_max = symmetry_adapt(Max_All_Params)
sym_adapt_lowconf = symmetry_adapt(Lowconf_All_Params)


Currently the sym_adapted_lowconf parameters are usable as-is. 

They include the minimum, maximum, and average values (FROM THE LOWEST ENERGY CONFORMER) for symmetry-equivalent groups. 

* The bolt, min, max, and lowconf are not yet ready to use. They currently give the minimum, maximum, and average values of the  boltz, min, max values across all conformers. This is inherently confusing so lets look at an example where we will specifically consider P1_NMR and P2_NMR for sym_adapt_min:


* Min_All_Params: populates each raw parameter with the lowest value across conformers. So P1_NMR would be the lowest measured P1_NMR and P2_NMR would be the lowest measured P2_NMR.


* When we convert this to sym_adpat_min, however, the script now looks at the minimum, maximum, and average values across symmetry equivalent groups in Min_All_Params. So for a C2 ligand, the minimum would be the minimum across P1 and P2 NMR, the maximum would be the maximum value of the two minimum P1 and P2 NMR values. The average and maximum values are therefore meaningless for sym_adapt_min and only the minimum value has physical meaning. 


* only the average is meaningful from sym_adapt_boltz


* only the max is meaningful from the sym_adapt_max


# Create a Dataframe For Symmetry Adapted Parameters from all conformers

* This will take the minimum values from sym_adapt_min, max values from sym_adapt_max, and average values for sym_adapt_boltz

* These paramters will be combined into a single dataframe: sym_adapt_allconf

In [18]:
# remove ligand Names from all but one of the min/max/boltz dataframes
del sym_adapt_max['Ligand']
del sym_adapt_boltz['Ligand']

# delete all but min paramters from sym_adapt_min dataframe, max paramters from sym_adapt_max dataframe,
# and avg paramters from sym_adapt_boltz dataframe 
for i in base_params:
    min_p, max_p, avg_p = i + '_min', i + '_max', i + '_avg'
    del sym_adapt_boltz[min_p]
    del sym_adapt_boltz[max_p]
    del sym_adapt_min[avg_p]
    del sym_adapt_min[max_p]
    del sym_adapt_max[min_p]
    del sym_adapt_max[avg_p]

# merge the min, max, and avg paramters into sym_adapt_allconf
sym_adapt_allconf = pd.merge(sym_adapt_min, sym_adapt_max, on='Ligand ID')
sym_adapt_allconf = pd.merge(sym_adapt_allconf, sym_adapt_boltz, on='Ligand ID')

# reorder the columns to match sym_adapt_lowconf
sym_adapt_allconf = sym_adapt_allconf[sym_adapt_lowconf.columns]
sym_adapt_allconf.head()

,Ligand,R1_NBO_min,R1_NBO_max,R1_NBO_avg,R2_NBO_min,R2_NBO_max,R2_NBO_avg,RBack1_NBO_min,RBack1_NBO_max,RBack1_NBO_avg,...,RATIO_N/S_octant_4.0_Ang_avg,RATIO_N/S_octant_5.0_Ang_min,RATIO_N/S_octant_5.0_Ang_max,RATIO_N/S_octant_5.0_Ang_avg,RATIO_N/S_octant_6.0_Ang_min,RATIO_N/S_octant_6.0_Ang_max,RATIO_N/S_octant_6.0_Ang_avg,RATIO_N/S_octant_7.0_Ang_min,RATIO_N/S_octant_7.0_Ang_max,RATIO_N/S_octant_7.0_Ang_avg
Ligand ID,,,,,,,,,,,,,,,,,,,,,
14,M003-1,-0.35858,-0.33016,-0.340155,-0.35595,-0.32723,-0.346805,-0.38822,-0.37800,-0.382882,...,1.067514,1.091723,1.165170,1.128446,1.163563,1.165032,1.164298,1.177067,1.211798,1.194432
16,W005-1,-0.35958,-0.32088,-0.358820,-0.37474,-0.35560,-0.374498,-0.33317,-0.32283,-0.327997,...,1.137767,0.912859,1.552432,1.232646,1.333404,1.524106,1.428755,1.399728,1.664697,1.532212
12,W001,-0.35659,-0.35077,-0.353206,-0.37225,-0.35796,-0.366268,-0.33308,-0.32699,-0.330529,...,1.464364,1.288023,1.889935,1.588979,2.034673,2.050251,2.042462,2.151146,2.822298,2.486722
10,SS_Et_Duphos,-0.51299,-0.48818,-0.498719,-0.48645,-0.47293,-0.480109,-0.35043,-0.34076,-0.345167,...,0.447142,0.336650,0.415055,0.375853,0.313314,0.393269,0.353291,0.313409,0.393135,0.353272
14_1,M003-1_ENANT,-0.35595,-0.32723,-0.346805,-0.35858,-0.33016,-0.340155,-0.38822,-0.37800,-0.382882,...,0.938477,0.858244,0.915984,0.887114,0.858346,0.859429,0.858887,0.825220,0.849569,0.837395


# Create All C2v Dataframe

In [19]:
def create_all_c2v(dataframe):
#sym_adapt_df is a dataframe that will be populated with the symmetry-adapted parameters for each ligand
    C2v_p_names = [[p + '_min', p + '_max', p + '_avg'] for p in list(geetum.C2v_dict.keys())]
    C2v_p_names = sum(C2v_p_names,[])
    all_c2v_df = pd.DataFrame(columns=['Ligand ID', 'Ligand'] + C2v_p_names + geetum.global_parameters)

    for index, row in dataframe.iterrows():
        if 'Error' in list(row):
            print(f"1+ error values were found in ligand {row['Ligand']} (ID: {row['Ligand ID']})")
            continue
        row_c2v = {'Ligand ID': row['Ligand ID'], 'Ligand': row['Ligand']}
        for p in geetum.global_parameters: #this keeps the values for parameters not affected by symmetry
            row_c2v[p] = row[p]
        C2v_dict = geetum.C2v_dict

        for c2v_param, params in C2v_dict.items():   #This updates parameters that are affected by symmetry to have min/max/avg vals 
            old_vals = [float(row[p]) for p in params] #This gives discrete values for symmetry-equivalent groups 
            
            p_min, p_max, p_avg = min(old_vals), max(old_vals), statistics.mean(old_vals)
            row_c2v[c2v_param + '_min'] = p_min
            row_c2v[c2v_param + '_max'] = p_max
            row_c2v[c2v_param + '_avg'] = p_avg

        all_c2v_df = pd.concat([all_c2v_df, pd.DataFrame(row_c2v, index=[0])], axis=0, ignore_index=True)

    all_c2v_df.set_index('Ligand ID', inplace=True)
    return(all_c2v_df)

all_c2v_lowconf = create_all_c2v(Lowconf_All_Params)
all_c2v_lowconf.head(10)


,Ligand,R_NBO_min,R_NBO_max,R_NBO_avg,RBack_NBO_min,RBack_NBO_max,RBack_NBO_avg,P_NMR_min,P_NMR_max,P_NMR_avg,...,bite_angle,NoPd_Homo,NoPd_Lumo,NoPd_dipole,pol,Vbur%_3.0_Ang,Vbur%_4.0_Ang,Vbur%_5.0_Ang,Vbur%_6.0_Ang,Vbur%_7.0_Ang
Ligand ID,,,,,,,,,,,,,,,,,,,,,
14,M003-1,-0.35594,-0.33016,-0.343077,-0.38822,-0.38294,-0.385580,226.1883,235.2270,230.70765,...,102.395428,-0.23134,-0.08159,4.0201,795.29,57.826468,58.316523,58.162586,57.212799,54.091570
16,W005-1,-0.37474,-0.32850,-0.350253,-0.49465,-0.32810,-0.411375,214.5602,239.3335,226.94685,...,97.091120,-0.20083,-0.07033,3.1133,716.98,63.706794,65.046785,63.394997,57.937888,50.120259
12,W001,-0.37225,-0.32925,-0.347915,-0.49678,-0.33308,-0.414930,211.7585,241.2173,226.48790,...,98.393806,-0.21170,-0.07060,4.0668,624.49,62.135913,63.715008,61.635158,56.091773,47.585433
10,SS_Et_Duphos,-0.51299,-0.47800,-0.490163,-0.34514,-0.34379,-0.344465,183.0535,189.8291,186.44130,...,88.758311,-0.17599,-0.03111,1.7517,342.79,57.540599,59.592437,56.425018,47.141594,33.130269
14_1,M003-1_ENANT,-0.35594,-0.33016,-0.343077,-0.38822,-0.38294,-0.385580,226.1883,235.2270,230.70765,...,102.395428,-0.23134,-0.08159,4.0201,795.29,57.826468,58.316523,58.162586,57.212799,54.091570
16_1,W005-1_ENANT,-0.37474,-0.32850,-0.350253,-0.49465,-0.32810,-0.411375,214.5602,239.3335,226.94685,...,97.091120,-0.20083,-0.07033,3.1133,716.98,63.706794,65.046785,63.394997,57.937888,50.120259
12_1,W001_ENANT,-0.37225,-0.32925,-0.347915,-0.49678,-0.33308,-0.414930,211.7585,241.2173,226.48790,...,98.393806,-0.21170,-0.07060,4.0668,624.49,62.135913,63.715008,61.635158,56.091773,47.585433
10_1,SS_Et_Duphos_ENANT,-0.51299,-0.47800,-0.490163,-0.34514,-0.34379,-0.344465,183.0535,189.8291,186.44130,...,88.758311,-0.17599,-0.03111,1.7517,342.79,57.540599,59.592437,56.425018,47.141594,33.130269


# Export Parameter Spreadsheets



In [21]:
# excel_filepath and excel_name are defined in section 2
writer = pd.ExcelWriter(excel_filepath + excel_name, engine='xlsxwriter')
all_c2v_lowconf.to_excel(writer, sheet_name='all_C2v_lowconf')
sym_adapt_lowconf.to_excel(writer, sheet_name='symm_adapt_lowconf')
writer.save()